# AI2
## This is the project No.2 and it's used to defeat opponent with min_max algorithm.

## Introduction
We have two player that are defined as YOU and CPU. In this project, We should add min_max function to use for choose a column for YOU that is better than the others to defeat CPU that is calculated by the heuristic that is defined below at final and change turn for CPU. I add some functions such as:
<ul>
    <li>
        <b>is_move_valid_curr_board</b>: is selected column valid or not
    </li>
    <li>    
        <b>get_possible_moves_curr_board</b>: get the columns that YOU can select and they have empty cell
    </li>        
    <li>        
        <b>check_if_player_has_won_curr_board</b>: check YOU or CPU satisfies the winning condition
    </li>        
    <li>        
        <b>is_terminal_node</b>: check that a player has won the match or the board has been fulled
    </li>       
    <li>       
        <b>evaluate_function</b>: assign a score for some conditions by the heuristic that define below
        <ul>
            <li>four cells YOU of four continuous rows, columns or diagonal cells gets 100 score</li>            
            <li>three cells YOU and one EMPTY of four continuous rows, columns or diagonal cells gets 50 score</li>
            <li>two cells YOU and two EMPTY of four continuous rows, columns or diagonal cells gets 15 score</li>
            <li>three cells CPU and one EMPTY of four continuous rows, columns or diagonal cells gets -80 score</li>
            <li>two cells CPU and two EMPTY of four continuous rows, columns or diagonal cells gets -40 score</li>            
        </ul>
    </li>
    <li>        
        <b>calculate_heuristic</b>: calculate heuristic for every four neighbor cells by evaluate_function<br/>
    </li>
    <li>        
        <b>get_empty_row</b>: find next row that has empty cell
    </li>     
    <li>        
        <b>minimax</b>: the algorithm to find best column according to the depth<br/>
Minimax is a decision rule for minimizing the possible loss for a worst case (maximum loss) scenario. When dealing with gains, it is referred to as "maximin"—to maximize the minimum gain. Originally formulated for n-player zero-sum game theory, covering both the cases where players take alternate moves and those where they make simultaneous moves, it has also been extended to more complex games and to general decision-making in the presence of uncertainty.<br/>The maximin value is the highest value that the player can be sure to get without knowing the actions of the other players; equivalently, it is the lowest value the other players can force the player to receive when they know the player's action.
    </li>       
</ul>



## Description

As described in the game introduction, in Connect4 the goal is to reach 4 beads of the same color in a horizontal, vertical or diagonal line. The first player to reach the above condition wins the game. If all the houses are filled and the condition of 4 houses with the same pieces does not happen, then the game is tied. The Connect4 game screen is a screen with 6 rows and 7 columns. Each player in each stage must choose one of the columns. The player piece is selected in the column until it reaches another piece or reaches the bottom row
He goes and gets there.

In [61]:
from random import random, choice
import copy
import math
import numpy as np
import pandas as pd
from time import time

In [62]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

    def run(self, depth, has_pruning, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [
            self.YOU, self.CPU], "starter value can only be 1,-1 or None"

        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):

            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input(depth, has_pruning)
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception(
                    "A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def is_move_valid_curr_board(self, move, table):
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return table[0][column_index] == 0

    def get_possible_moves_curr_board(self, table):
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid_curr_board(move, table)):
                move -= 1
                possible_moves.append(move)
        return possible_moves

    def check_if_player_has_won_curr_board(self, table, player_id):

        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if table[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True

        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if table[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True

        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if table[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if table[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True

    def is_terminal_node(self, table):
        return self.check_if_player_has_won_curr_board(table, self.YOU) or self.check_if_player_has_won_curr_board(table, self.CPU) or len(self.get_possible_moves_curr_board(table)) == 0

    def evaluate_function(self, neighbor_pieces, player_id):
        score = 0
        if player_id == self.YOU:
            opp_id = self.CPU
        else:
            opp_id = self.YOU

        if neighbor_pieces.count(player_id) == 4:
            score += 100
        elif neighbor_pieces.count(player_id) == 3 and neighbor_pieces.count(self.EMPTY) == 1:
            score += 50
        elif neighbor_pieces.count(player_id) == 2 and neighbor_pieces.count(self.EMPTY) == 2:
            score += 15

        if neighbor_pieces.count(opp_id) == 3 and neighbor_pieces.count(self.EMPTY) == 1:
            score -= 80
        elif neighbor_pieces.count(opp_id) == 2 and neighbor_pieces.count(self.EMPTY) == 2:
            score -= 40

        return score

    def calculate_heuristic(self, table, player_id):
        score = 0

        for r in range(self.rows):
            row_array = table[r][:]
            for c in range(self.columns-3):
                score += self.evaluate_function(row_array[c:c+4], player_id)

        for c in range(self.columns):
            col_array = [i[c] for i in table]
            for r in range(self.rows-3):
                score += self.evaluate_function(col_array[r:r+4], player_id)

        for r in range(self.rows-3):
            for c in range(self.columns-3):
                neighbor_pieces = [table[r+i][c+i] for i in range(4)]
                score += self.evaluate_function(neighbor_pieces, player_id)

        for r in range(self.rows-3):
            for c in range(self.columns-3):
                neighbor_pieces = [table[r+3-i][c+i] for i in range(4)]
                score += self.evaluate_function(neighbor_pieces, player_id)

        return score

    def get_empty_row(self, table, col):
        for r in reversed(range(self.rows)):
            if table[r][col] == 0:
                return r

    def minimax(self, table, depth, alpha, beta, maximizingPlayer, has_pruning):
        valid_locations = self.get_possible_moves_curr_board(table)
        is_terminal = self.is_terminal_node(table)
        if depth == 0 or is_terminal:
            if is_terminal:
                if self.check_if_player_has_won_curr_board(table, self.CPU):
                    return (None, -100000000000000)
                elif self.check_if_player_has_won_curr_board(table, self.YOU):
                    return (None, 10000000000000)
                else:  # Game is over, no more valid moves
                    return (None, 0)
            else:  # Depth is zero
                return (None, self.calculate_heuristic(table, self.YOU))
        if maximizingPlayer:
            value = -math.inf
            column = 0
            for col in valid_locations:
                row = self.get_empty_row(table, col)
                b_copy = copy.deepcopy(table)
                b_copy[row][col] = self.YOU
                new_score = self.minimax(
                    b_copy, depth-1, alpha, beta, False, has_pruning)[1]
                if new_score > value:
                    value = new_score
                    column = col
                if has_pruning:
                    alpha = max(alpha, value)
                    if alpha >= beta:
                        break
            return column, value

        else:  # Minimizing player
            value = math.inf
            column = 0
            for col in valid_locations:
                row = self.get_empty_row(table, col)
                b_copy = copy.deepcopy(table)
                b_copy[row][col] = self.CPU
                new_score = self.minimax(
                    b_copy, depth-1, alpha, beta, True, has_pruning)[1]
                if new_score < value:
                    value = new_score
                    column = col
                if has_pruning:
                    beta = min(beta, value)
                    if alpha >= beta:
                        break
            return column, value

    def get_your_input(self, depth, has_pruning):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        # TODO: complete here
        b_copy = copy.deepcopy(self.board)
        col, minimax_score = self.minimax(
            b_copy, depth, -math.inf, math.inf, True, has_pruning)
        col += 1
        return col

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )

    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0

    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves

    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]

    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]

    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True

    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")

    def __change_turn(self, turn):
        if (turn == self.YOU):
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent):
            return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"

                print(f"{house_char}", end=" ")
            print()

## Calculate results
Calculate results for testcases that is being come below. Run the game for 20 times and calculate the percent of winning and the time is spended for games

In [63]:
board_sizes_to_check = [(6, 7),
                        (7, 8),
                        (7, 10)]


def play_game(depth, has_pruning, board):
    start = time()
    game = ConnectSin(board_size=board, silent=True)
    results = {-1: 0, 0: 0, 1: 0}
    
    for i in range(20):
        results[game.run(depth, has_pruning)] += 1
    win_percent = (results[1] / (results[1] + results[0] + results[-1])) * 100
    
    finish = time()
    spended_time = finish - start
    return [spended_time, win_percent]


## Testcase 1 with board (6, 7)

In [64]:
res = []
board = board_sizes_to_check[0]
res.append(play_game(1, True, board))
res.append(play_game(1, False, board))
res.append(play_game(3, True, board))
res.append(play_game(3, False, board))
res.append(play_game(5, True, board))
res.append(play_game(5, False, board))

df = pd.DataFrame(res, index=['depth=1 and has pruning ', 'depth=1 and without pruning ', 'depth=3 and has pruning ', 'depth=3 and without pruning ', 'depth=5 and has pruning ', 'depth=5 and without pruning '] ,columns=[f'spended time for {board}', 'win percent'])

df

,"spended time for (6, 7)",win percent
depth=1 and has pruning,0.776142,100.0
depth=1 and without pruning,0.969452,100.0
depth=3 and has pruning,8.747520,95.0
depth=3 and without pruning,21.865705,95.0
depth=5 and has pruning,127.337225,100.0
depth=5 and without pruning,785.231513,100.0


## Testcase 1 with board (7, 8)

In [66]:
res = []
board = board_sizes_to_check[1]
res.append(play_game(1, True, board))
res.append(play_game(1, False, board))
res.append(play_game(3, True, board))
res.append(play_game(3, False, board))
res.append(play_game(5, True, board))
res.append(play_game(5, False, board))

df = pd.DataFrame(res, index=['depth=1 and has pruning ', 'depth=1 and without pruning ', 'depth=3 and has pruning ', 'depth=3 and without pruning ', 'depth=5 and has pruning ', 'depth=5 and without pruning '] ,columns=[f'spended time for {board}', 'win percent'])

df

,"spended time for (7, 8)",win percent
depth=1 and has pruning,1.512789,95.0
depth=1 and without pruning,1.005325,90.0
depth=3 and has pruning,16.894001,100.0
depth=3 and without pruning,41.757502,100.0
depth=5 and has pruning,287.717486,100.0
depth=5 and without pruning,1093.574246,100.0


## Testcase 1 with board (7, 10)

In [67]:
res = []
board = board_sizes_to_check[2]
res.append(play_game(1, True, board))
res.append(play_game(1, False, board))
res.append(play_game(3, True, board))
res.append(play_game(3, False, board))
res.append(play_game(5, True, board))
res.append(play_game(5, False, board))

df = pd.DataFrame(res, index=['depth=1 and has pruning ', 'depth=1 and without pruning ', 'depth=3 and has pruning ', 'depth=3 and without pruning ', 'depth=5 and has pruning ', 'depth=5 and without pruning '] ,columns=[f'spended time for {board}', 'win percent'])

df

,"spended time for (7, 10)",win percent
depth=1 and has pruning,0.362359,95.0
depth=1 and without pruning,0.345077,95.0
depth=3 and has pruning,8.602568,100.0
depth=3 and without pruning,27.726939,100.0
depth=5 and has pruning,158.170102,100.0
depth=5 and without pruning,2188.836160,100.0


## Questions

Q1: I implement this in heuristic function that it assigns scores to some conditons; for example add 100 to score if 3 neighbor cells are YOU and another one is EMPTY. This help YOU to choose better cell that can be closer to defeat opponent or to avoid CPU to win the game. In this heuristic, we can find the best move that is more closer to win the game and also it avoid the best move of CPU and that's why I choose this heuristic. <br>
Q2: If the depth is higher, then YOU's winning probability is higher but it spends more time; because it sees more nodes and calculate more heuristic values and the result will be more accurate. Also I ran the game for 200 times for 1 and 3 depth and the probability of winning is being higher for bigger depth.<br>
Q3: The columns check out one by one from 1 to column numbers but when it compares alpha and beta with value and if it satisfies the condition, it will not see remain columns. This helps to jump some nodes that it doesn't need to see.